In [1]:
using Pkg
Pkg.develop(path="../SNNT")
Pkg.resolve()
Pkg.precompile()

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
Precompiling project...
  ✓ SNNT
  ✓ ExpertWaffle
  2 dependencies successfully precompiled in 63 seconds (351 already precompiled)


In [2]:
using Revise

In [3]:
using SNNT

┌ Info: Precompiling SNNT [7b70dde2-2dce-46fe-8bfa-c0ba9c54c2e7]
└ @ Base loading.jl:1423
┌ Warning: Package SNNT does not have LinearAlgebra in its dependencies:
│ - If you have SNNT checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with SNNT
└ Loading LinearAlgebra into SNNT from project dependency, future warnings for SNNT are suppressed.


In [4]:
using SymbolicUtils

In [5]:
fixed_vars=Dict{String,Union{String,Number}}("T"=>0.1,"cpost"=>0.0,"A"=>100,"B"=>100,"rVelpost"=>"rVel","rPospost"=>"rPos")
mapping=Dict{String,Tuple{SNNT.AST.VariableType,Int64}}("rPos"=>(SNNT.AST.Input,1),"rVel"=>(SNNT.AST.Input,2),"rAccpost"=>(SNNT.AST.Output,1))

Dict{String, Tuple{SNNT.AST.VariableType, Int64}} with 3 entries:
  "rAccpost" => (Output, 1)
  "rVel"     => (Input, 2)
  "rPos"     => (Input, 1)

In [8]:
@time test=load_query("../SNNT/test/parsing/examples/example3",fixed_vars,mapping)

  0.278166 seconds (1.56 M allocations: 80.924 MiB, 17.28% gc time)


SNNT.AST.Query((((Rational{Int128}[-1//1, 0//1, 0//1]<=0//1&Rational{Int128}[1//1, 0//1, 0//1]<=100//1&Rational{Int128}[0//1, -1//1, 0//1]<=200//1&Rational{Int128}[0//1, 1//1, 0//1]<=200//1&Rational{Int128}[0//1, 0//1, -1//1]<=100//1&Rational{Int128}[0//1, 0//1, 1//1]<=100//1)&(Rational{Int128}[-1//1, 0//1, 0//1]<0//1&((-1.0*rPos)+(0.005*(rVel^2.0)))<=0.0))->((Rational{Int128}[0//1, 0//1, 1//1]<=100//1&Rational{Int128}[0//1, 0//1, -1//1]<=-100//1)|(Rational{Int128}[0//1, 0//1, -1//1]<=100//1&Rational{Int128}[0//1, 0//1, 1//1]<100//1&(Rational{Int128}[0//1, 0//1, 1//1]<0//1|Rational{Int128}[0//1, 0//1, -1//1]<0//1)&(((Rational{Int128}[0//1, 1//1, 1//10]<0//1|Rational{Int128}[0//1, -1//1, 0//1]<0//1)&((-1.0*rPos)+(5.0e-5*(rAccpost^2.0))+(-0.005*rAccpost)+(0.005*(rVel^2.0))+(-0.1*rVel)+(0.001*rAccpost*rVel))<0.0)|(((-1.0*rPos)+(5.0e-5*(rAccpost^2.0))+(-0.005*rAccpost)+(0.005*(rVel^2.0))+(-0.1*rVel)+(0.001*rAccpost*rVel))<0.0&((0.5*(rVel^2.0))+(-1.0*rAccpost*rPos))<0.0)))|(((-1.0*rPos)+(0.

In [9]:
@time test2=SNNT.Control.prepare_for_olnnv(test)

  0.000107 seconds (239 allocations: 11.234 KiB)


SNNT.AST.Query(!((((Rational{Int128}[-1//1, 0//1, 0//1]<=0//1&Rational{Int128}[1//1, 0//1, 0//1]<=100//1&Rational{Int128}[0//1, -1//1, 0//1]<=200//1&Rational{Int128}[0//1, 1//1, 0//1]<=200//1&Rational{Int128}[0//1, 0//1, -1//1]<=100//1&Rational{Int128}[0//1, 0//1, 1//1]<=100//1)&(Rational{Int128}[-1//1, 0//1, 0//1]<0//1&O(((-1.0*rPos)+(0.005*(rVel^2.0)))<=0.0)))->((Rational{Int128}[0//1, 0//1, 1//1]<=100//1&Rational{Int128}[0//1, 0//1, -1//1]<=-100//1)|(Rational{Int128}[0//1, 0//1, -1//1]<=100//1&Rational{Int128}[0//1, 0//1, 1//1]<100//1&(Rational{Int128}[0//1, 0//1, 1//1]<0//1|Rational{Int128}[0//1, 0//1, -1//1]<0//1)&(((Rational{Int128}[0//1, 1//1, 1//10]<0//1|Rational{Int128}[0//1, -1//1, 0//1]<0//1)&U(((-1.0*rPos)+(5.0e-5*(rAccpost^2.0))+(-0.005*rAccpost)+(0.005*(rVel^2.0))+(-0.1*rVel)+(0.001*rAccpost*rVel))<0.0))|(U(((-1.0*rPos)+(5.0e-5*(rAccpost^2.0))+(-0.005*rAccpost)+(0.005*(rVel^2.0))+(-0.1*rVel)+(0.001*rAccpost*rVel))<0.0)&U(((0.5*(rVel^2.0))+(-1.0*rAccpost*rPos))<0.0))))|(U(

In [57]:
ENV["JULIA_DEBUG"] = nothing

In [56]:
i = 0
@time SNNT.Control.run_query(test2) do linear_term
    global i+=1
    println("---------------------------------------------------------------------------------------------")
    println("Bounds:", linear_term[1])
    println("# Linear Constraints: ",length(linear_term[1]))
    println("Mixed Conjunctions ",linear_term[2])
end
println("Number of linear queries: ",i)

┌ Debug: Atom or LinearConstraint => constraint variable
└ @ SNNT.QueryGeneration /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/QueryGeneration/Skeleton.jl:14
┌ Debug: Atom or LinearConstraint => constraint variable
└ @ SNNT.QueryGeneration /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/QueryGeneration/Skeleton.jl:14
┌ Debug: Atom or LinearConstraint => constraint variable
└ @ SNNT.QueryGeneration /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/QueryGeneration/Skeleton.jl:14
┌ Debug: Atom or LinearConstraint => constraint variable
└ @ SNNT.QueryGeneration /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/QueryGeneration/Skeleton.jl:14
┌ Debug: Atom or LinearConstraint => constraint variable
└ @ SNNT.QueryGeneration /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/QueryGeneration/Skeleton.jl:14
┌ Debug: Atom or LinearConstraint => constraint variable
└ @ SNNT.QueryGene

┌ Debug: Generating semilinear constraint...
└ @ SNNT.Analysis /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Analysis/ProcessConstraints.jl:152
┌ Debug: ("Adding in-out constraints: ", Tuple{Int64, Union{SNNT.AST.LinearConstraint, SNNT.AST.SemiLinearConstraint}}[(5, Rational{Int128}[0//1, 0//1, -1//1]<=100//1), (6, Rational{Int128}[0//1, 0//1, 1//1]<=100//1), (-12, Rational{Int128}[0//1, 0//1, 1//1]<100//1), (14, Rational{Int128}[0//1, 0//1, 1//1]<100//1), (-15, Rational{Int128}[0//1, 0//1, -1//1]<=0//1), (-16, Rational{Int128}[0//1, 0//1, 1//1]<=0//1), (18, Rational{Int128}[0//1, 1//1, 1//10]<0//1), (-21, Rational{Int128}[1//1, 1//10, 5629499534214//1125899906842799]+N(3)<=0//1), (-23, Rational{Int128}[0//1, 0//1, 0//1]+N(2)<=0//1), (28, Rational{Int128}[0//1, 0//1, 1//1]<=0//1), (29, Rational{Int128}[0//1, 0//1, -1//1]<=0//1)])
└ @ SNNT.QueryGeneration /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/QueryGeneration/Iterator.jl:150
┌ D

┌ Info: Simplifying
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:282
┌ Info: Resolving univariate
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:284
┌ Info: Simplifying
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:286
┌ Info: Resolving multivariate
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:290
┌ Info: Simplifying
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:282
┌ Info: Resolving univariate
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:284
┌ Info: Simplifying
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:286
┌ Info: Resolving m

┌ Debug: ("Coefficients after: ", Float32[1.0, 2.2090604, 0.22406033])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:47
┌ Debug: Adding nonlinear constraint
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:94
┌ Debug: ("Linear term: ", Rational{Int128}[-154018043//6308435, 0//1, 324441287//8584763]+4068400790//986727)
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:44
┌ Debug: ("Coefficients before: ", Float32[0.0, 0.0, 0.0])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:45
┌ Debug: ("Coefficients after: ", Float32[-24.414621, 0.0, 37.79269])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:47
┌ Debug: ("Linear term: ", Rational{Int128}[0//1, -400//1, 0//1]+-40400//1)
└ @ SNNT.Approx /home/steu

┌ Debug: ("Linear term: ", Rational{Int128}[-154018043//6308435, 0//1, 324441287//8584763]+4068400790//986727)
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:44
┌ Debug: ("Coefficients before: ", Float32[0.0, 0.0, 0.0])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:45
┌ Debug: ("Coefficients after: ", Float32[-24.414621, 0.0, 37.79269])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:47
┌ Debug: ("Linear term: ", Rational{Int128}[0//1, -400//1, 0//1]+-40400//1)
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:44
┌ Debug: ("Coefficients before: ", Float32[-24.414621, 0.0, 37.79269])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:45
┌ Debug: ("Coefficients after: ", Float32[-24.414621, 200.0, 37

...

┌ Debug: ("OUTPUT with ", 13, " constraints (", 11, " linear, ", 2, " nonlinear)")
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:80
┌ Debug: Adding nonlinear constraint
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:94
┌ Debug: ("Linear term: ", Rational{Int128}[0//1, -1556647617//34593929, -1179764201//6482950]+-3543561165//946442)
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:44
┌ Debug: ("Coefficients before: ", Float32[-1.0, -0.1, -0.005])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:45
┌ Debug: ("Coefficients after: ", Float32[-1.0, -0.14499772, -0.18697955])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:47
┌ Debug: ("Linear term: ", Rational{Int128}[0//1, 0//1, -400//3]+-29200//9)

Excessive output truncated after 524307 bytes.

Using N=2, ϵ=0.01
Using N=2, ϵ=0.01
Using N=2, ϵ=0.01
Using N=2, ϵ=0.01


┌ Debug: ("Linear term: ", Rational{Int128}[0//1, -504971130//10450669, -938516275//6327666]+-7639905301//606992)
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:44
┌ Debug: ("Coefficients before: ", Float32[1.0, 0.1, 0.005])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:45
┌ Debug: ("Coefficients after: ", Float32[1.0, 0.14831951, 0.15331951])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:47
┌ Debug: ("Linear term: ", Rational{Int128}[0//1, 0//1, -200//1]+-10100//1)
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:44
┌ Debug: ("Coefficients before: ", Float32[1.0, 0.14831951, 0.15331951])
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Iterator.jl:45
┌ Debug: ("Coefficients after: ", Float32[1.0, 0.14831

LoadError: Failed to apply rule ~(x::not_division) * +(~(~y)) => +((distribute_factor(x, y)...)) on expression 1.6730024*((-1.8017315+(-0.028017312*x3)))

In [59]:
i = 0
@time SNNT.Control.run_query(test2) do linear_term
    global i+=1
    println("---------------------------------------------------------------------------------------------")
    println("Bounds:", linear_term[1])
    println("# Linear Constraints: ",length(linear_term[2]))
    println("# Mixed Conjunctions ",length(linear_term[3]))
end
println("Number of linear queries: ",i)

Using N=2, ϵ=0.01
Using N=2, ϵ=0.01
Using N=2, ϵ=0.01
Using N=2, ϵ=0.01
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-200.001, -182.23540399556057)]
# Linear Constraints: 2
# Mixed Conjunctions 45
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-182.23740399556058, -164.30678497588076)]
# Linear Constraints: 2
# Mixed Conjunctions 45
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-164.30878497588077, -149.999)]
# Linear Constraints: 2
# Mixed Conjunctions 45
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-150.001, -133.33233333333334)]
# Linear Constraints: 2
# Mixed Conjunctions 45
---------------------------------------------------------------------

Bounds:[(-0.001, 8.882796748823568), (-0.001, 17.7645960044395)]
# Linear Constraints: 2
# Mixed Conjunctions 7
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (17.762596004439498, 35.69321065909807)]
# Linear Constraints: 2
# Mixed Conjunctions 7
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (35.691210659098076, 50.001)]
# Linear Constraints: 2
# Mixed Conjunctions 7
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (49.999, 66.66766666666668)]
# Linear Constraints: 2
# Mixed Conjunctions 7
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (66.66566666666667, 99.60306337509212)]
# Linear Constraints: 2
# Mixed Conjunctions 7
------------------------------------

---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-164.30878934090197, -149.999)]
# Linear Constraints: 2
# Mixed Conjunctions 7
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-150.001, -133.33233333333334)]
# Linear Constraints: 2
# Mixed Conjunctions 7
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-133.33433333333335, -100.39693662490788)]
# Linear Constraints: 2
# Mixed Conjunctions 7
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-100.39893662490789, -88.68495952689783)]
# Linear Constraints: 2
# Mixed Conjunctions 7
---------------------------------------------------------------------------------------------
Bounds:[(-0.001, 8.882796748823568), (-88

┌ Info: Simplifying
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:282
┌ Info: Resolving univariate
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:284
┌ Info: Simplifying
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:286
┌ Info: Resolving multivariate
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:290
┌ Info: Simplifying
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:282
┌ Info: Resolving univariate
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:284
┌ Info: Simplifying
└ @ SNNT.Approx /home/steuber/Dokumente/Hortus/Univ/KIT/Master/Work/repos/SNNT/SNNT/src/Approx/Approximation.jl:286
┌ Info: Resolving m